In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import re

def parse_standard_listing(item):

    try:
        title_element = item.find('p', class_='content__list--item--title')
        if not title_element or not title_element.find('a'):
            return None 

        detail_link = "贝壳租房网站" + title_element.find('a')['href']
        # 放弃品牌公寓类型数据的爬取
        if '/apartment/' in detail_link:
            return None

        listing_data = {
            '标题': 'N/A', '价格(元/月)': 0, '区域': 'N/A',
            '户型': 'N/A', '面积(㎡)': 'N/A', '朝向': 'N/A', '详情链接': 'N/A'
        }
        listing_data['标题'] = title_element.get_text(strip=True)
        listing_data['详情链接'] = detail_link

        price_element = item.find('span', class_='content__list--item-price')
        if price_element and price_element.find('em'):
            price_text = price_element.find('em').get_text(strip=True)
            listing_data['价格(元/月)'] = int(price_text) if price_text.isdigit() else 0
        else:
            return None 

        des_element = item.find('p', class_='content__list--item--des')
        if des_element:
            full_des_text = des_element.get_text(strip=True)
            des_parts = [part.strip() for part in full_des_text.split('/')]
            
            if des_parts:
                location_info = des_parts.pop(0).replace('·', ' - ')
                listing_data['区域'] = location_info

            for part in des_parts:
                if '㎡' in part:
                    listing_data['面积(㎡)'] = re.sub(r'\s*㎡', '', part)
                elif '室' in part or '厅' in part or '卫' in part:
                    listing_data['户型'] = part
                elif len(part) <= 2 and any(d in part for d in ['东', '南', '西', '北']):
                    listing_data['朝向'] = part
        
        return listing_data

    except Exception:
        return None

def get_beike_rent_info_final_v2(max_pages=5):
    base_url = "贝壳租房网站"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    }
    
    all_listings = []
    print(f"--- 自动跳过所有品牌公寓房源 ---")

    for page in range(1, max_pages + 1):
        url = f"{base_url}pg{page}/"
        print(f"正在爬取第 {page} 页: {url}")

        try:
            response = requests.get(url, headers=headers, timeout=15)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'lxml')
            
            listings_on_page = soup.find_all('div', class_='content__list--item')
            if not listings_on_page:
                print(f"警告: 在第 {page} 页没有找到房源信息，爬取提前结束。")
                break

            count_on_page = 0
            for item in listings_on_page:
                listing_data = parse_standard_listing(item)
                if listing_data:
                    all_listings.append(listing_data)
                    count_on_page += 1
            
            print(f"第 {page} 页成功解析 {count_on_page} 条【普通房源】。")

            sleep_time = random.uniform(2, 4)
            print(f"休眠 {sleep_time:.2f} 秒...")
            time.sleep(sleep_time)

        except requests.exceptions.RequestException as e:
            print(f"请求第 {page} 页时发生网络错误: {e}")
            break
            
    print("--- 爬取结束 ---")
    
    if all_listings:
        return pd.DataFrame(all_listings)
    else:
        return pd.DataFrame()

if __name__ == '__main__':
    PAGES_TO_SCRAPE = 20 #可以把数字改成自己想爬取的页数

    rent_data_df = get_beike_rent_info_final_v2(max_pages=PAGES_TO_SCRAPE)

    if not rent_data_df.empty:
        try:
            output_filename = '文件保存位置'
            columns_order = ['标题', '价格(元/月)', '区域', '户型', '面积(㎡)', '朝向', '详情链接']
            rent_data_df = rent_data_df.reindex(columns=columns_order) 
            
            rent_data_df.to_csv(output_filename, index=False, encoding='utf-8-sig')
            
            print(f"\n成功爬取 {len(rent_data_df)} 条【普通房源】信息。")
            print(f"数据已保存到文件: {output_filename}")
            print("\n数据预览:")
            print(rent_data_df.head())
        except Exception as e:
            print(f"保存文件时出错: {e}")
    else:
        print("\n未能爬取到任何有效的普通房源信息，程序结束。")

--- 【再次修正版】开始爬取，将自动跳过所有“品牌公寓”房源 ---
正在爬取第 1 页: https://nn.zu.ke.com/zufang/pg1/
第 1 页成功解析 17 条【普通房源】。
休眠 2.29 秒...
正在爬取第 2 页: https://nn.zu.ke.com/zufang/pg2/
第 2 页成功解析 17 条【普通房源】。
休眠 3.81 秒...
正在爬取第 3 页: https://nn.zu.ke.com/zufang/pg3/
第 3 页成功解析 17 条【普通房源】。
休眠 3.52 秒...
正在爬取第 4 页: https://nn.zu.ke.com/zufang/pg4/
第 4 页成功解析 18 条【普通房源】。
休眠 3.80 秒...
正在爬取第 5 页: https://nn.zu.ke.com/zufang/pg5/
第 5 页成功解析 30 条【普通房源】。
休眠 2.58 秒...
正在爬取第 6 页: https://nn.zu.ke.com/zufang/pg6/
第 6 页成功解析 30 条【普通房源】。
休眠 2.59 秒...
正在爬取第 7 页: https://nn.zu.ke.com/zufang/pg7/
第 7 页成功解析 22 条【普通房源】。
休眠 2.53 秒...
正在爬取第 8 页: https://nn.zu.ke.com/zufang/pg8/
第 8 页成功解析 22 条【普通房源】。
休眠 2.80 秒...
正在爬取第 9 页: https://nn.zu.ke.com/zufang/pg9/
第 9 页成功解析 23 条【普通房源】。
休眠 3.03 秒...
正在爬取第 10 页: https://nn.zu.ke.com/zufang/pg10/
第 10 页成功解析 22 条【普通房源】。
休眠 2.72 秒...
正在爬取第 11 页: https://nn.zu.ke.com/zufang/pg11/
第 11 页成功解析 18 条【普通房源】。
休眠 2.11 秒...
正在爬取第 12 页: https://nn.zu.ke.com/zufang/pg12/
第 12 页成功解析 18 条【普通房源】。
休眠 3.26 秒...
正在爬取第 1

In [ ]:
import pandas as pd
import requests
import time
import random


GAODE_API_KEY = "你的api" 


COMPANY_ADDRESS = "公司地址" 

INPUT_CSV = '读取的文件'
OUTPUT_CSV = '输出的文件' 


def get_coordinates(address, api_key):
    geocode_url = "https://restapi.amap.com/v3/geocode/geo"
    params = {'key': api_key, 'address': address, 'city': '你爬取的城市'}
    try:
        response = requests.get(geocode_url, params=params, timeout=10)
        data = response.json()
        if data['status'] == '1' and data['geocodes']:
            return data['geocodes'][0]['location']
    except Exception as e:
        print(f"  地址解析API请求失败: {e}")
    return None

def get_drive_info(origin, destination, api_key):
    url = "https://restapi.amap.com/v3/direction/driving"
    params = {'key': api_key, 'origin': origin, 'destination': destination}
    try:
        response = requests.get(url, params=params, timeout=10)
        data = response.json()
        if data['status'] == '1' and data.get('route', {}).get('paths'):
            path = data['route']['paths'][0]
            distance_km = round(int(path['distance']) / 1000, 1)
            duration_min = round(int(path['duration']) / 60)
            return f"{distance_km}公里", f"{duration_min}分钟"
    except Exception:
        pass
    return "计算失败", "计算失败"

def get_transit_info(origin, destination, api_key):
    url = "https://restapi.amap.com/v3/direction/transit/integrated"
    params = {
        'key': api_key,
        'origin': origin,
        'destination': destination,
        'city': '你所爬取的城市', 
        'strategy': '0' 
    }
    try:
        response = requests.get(url, params=params, timeout=10)
        data = response.json()
        if data['status'] == '1' and data.get('route', {}).get('transits'):
            transit = data['route']['transits'][0]
            duration_min = round(int(transit['duration']) / 60)
            return f"{duration_min}分钟"
    except Exception:
        pass
    return "计算失败"

def get_walking_info(origin, destination, api_key):
    url = "https://restapi.amap.com/v3/direction/walking"
    params = {'key': api_key, 'origin': origin, 'destination': destination}
    try:
        response = requests.get(url, params=params, timeout=10)
        data = response.json()
        if data['status'] == '1' and data.get('route', {}).get('paths'):
            path = data['route']['paths'][0]
            distance_km = round(int(path['distance']) / 1000, 1)
            duration_min = round(int(path['duration']) / 60)
            return f"{distance_km}公里", f"{duration_min}分钟"
    except Exception:
        pass
    return "计算失败", "计算失败"

def main():
    if GAODE_API_KEY == "在这里粘贴你从高德开放平台获取的Key":
        print("错误：请先在代码中配置你的高德API Key！")
        return

    try:
        df = pd.read_csv(INPUT_CSV)
        print(f"成功读取 {len(df)} 条房源数据自 '{INPUT_CSV}'。")
    except FileNotFoundError:
        print(f"错误：找不到输入文件 '{INPUT_CSV}'。")
        return

    print(f"正在获取公司地址 '{COMPANY_ADDRESS}' 的坐标...")
    company_coords = get_coordinates(COMPANY_ADDRESS, GAODE_API_KEY)
    if not company_coords:
        print("无法获取公司坐标，程序终止。")
        return
    print(f"公司坐标获取成功: {company_coords}")

    new_columns_data = []
    total = len(df)
    for index, row in df.iterrows():
        rent_address = f"你所爬取的城市{row['区域'].replace(' - ', '')}"
        print(f"\n({index + 1}/{total}) 正在处理: {row['标题']}")
        
        commute_info = {
            '驾车距离': '获取坐标失败', '驾车时间': '获取坐标失败',
            '公交时间': '获取坐标失败', '步行距离': '获取坐标失败', '步行时间': '获取坐标失败'
        }

        rent_coords = get_coordinates(rent_address, GAODE_API_KEY)
        
        if rent_coords:
            drive_dist, drive_time = get_drive_info(rent_coords, company_coords, GAODE_API_KEY)
            commute_info['驾车距离'] = drive_dist
            commute_info['驾车时间'] = drive_time
            print(f"  驾车: {drive_dist}, {drive_time}")

            transit_time = get_transit_info(rent_coords, company_coords, GAODE_API_KEY)
            commute_info['公交时间'] = transit_time
            print(f"  公交: {transit_time}")

            walk_dist, walk_time = get_walking_info(rent_coords, company_coords, GAODE_API_KEY)
            commute_info['步行距离'] = walk_dist
            commute_info['步行时间'] = walk_time
            print(f"  步行: {walk_dist}, {walk_time}")
        
        new_columns_data.append(commute_info)

        time.sleep(random.uniform(0.1, 0.3)) #控制频率

    commute_df = pd.DataFrame(new_columns_data, index=df.index)
    df_final = pd.concat([df, commute_df], axis=1)

    try:
        df_final.to_csv(OUTPUT_CSV, index=False, encoding='utf-8-sig')
        print(f"\n--- 处理完成！---")
        print(f"已将包含多种出行方式的结果保存到新文件: '{OUTPUT_CSV}'")
        print("\n最终数据预览:")
        preview_cols = ['标题', '价格(元/月)', '驾车时间', '公交时间', '步行时间']
        print(df_final[preview_cols].head())
    except Exception as e:
        print(f"保存文件时出错: {e}")

if __name__ == '__main__':
    main()

成功读取 414 条房源数据自 'E:\xiangmu\zufang\beike_rent_nn.csv'。
正在获取公司地址 '南宁市兴宁区朝阳路38号' 的坐标...
公司坐标获取成功: 108.321461,22.816342

(1/414) 正在处理: 整租·骋望珺玺 3室2厅 南
  驾车: 9.7公里, 21分钟
  公交: 50分钟
  步行: 8.4公里, 112分钟

(2/414) 正在处理: 整租·中海哈罗学府 3室2厅 南
  驾车: 19.9公里, 44分钟
  公交: 76分钟
  步行: 19.4公里, 258分钟

(3/414) 正在处理: 整租·广西壮族自治区疾病预防控制中心宿舍区 4室2厅 南
  驾车: 3.6公里, 12分钟
  公交: 34分钟
  步行: 3.0公里, 40分钟

(4/414) 正在处理: 整租·龙滩苑 3室2厅 南
  驾车: 8.1公里, 33分钟
  公交: 24分钟
  步行: 7.6公里, 102分钟

(5/414) 正在处理: 整租·君华•锦云 4室2厅 北
  驾车: 18.3公里, 47分钟
  公交: 67分钟
  步行: 18.1公里, 241分钟

(6/414) 正在处理: 整租·大唐盛世观湖 4室2厅 南
  驾车: 20.4公里, 44分钟
  公交: 87分钟
  步行: 19.8公里, 264分钟

(7/414) 正在处理: 整租·永凯春晖花园西C区 3室2厅 南
  驾车: 8.1公里, 35分钟
  公交: 30分钟
  步行: 8.0公里, 107分钟

(8/414) 正在处理: 整租·五象澜庭府锦苑 3室2厅 东南/南
  驾车: 15.3公里, 43分钟
  公交: 46分钟
  步行: 14.6公里, 194分钟

(9/414) 正在处理: 整租·恒悦居 3室2厅 南/北
  驾车: 21.1公里, 49分钟
  公交: 60分钟
  步行: 14.6公里, 195分钟

(10/414) 正在处理: 整租·万科紫台 4室3厅 南
  驾车: 11.7公里, 40分钟
  公交: 45分钟
  步行: 10.8公里, 145分钟

(11/414) 正在处理: 整租·宏桂东盛 3室2厅 南
  驾车: 11.1公里, 39分钟
  公交: 50分钟


In [ ]:
import pandas as pd
import requests
import time
import folium
from folium.plugins import FeatureGroupSubGroup, MarkerCluster 

GAODE_API_KEY = "你的api" 
COMPANY_ADDRESS = "公司地址"
WEIGHTS = {'price': 0.4, 'commute': 0.4, 'area': 0.2}
INPUT_CSV = '读取的数据'
OUTPUT_CSV_SCORED = '保存的数据' 
OUTPUT_MAP_HTML = '保存的地图'



def get_coordinates_final(area_string, api_key):

    parts = area_string.split(' - ')
    keyword = parts[-1] 
    

    lat, lon = search_poi_by_keyword(keyword, api_key)
    if lat:
        return lat, lon


    lat, lon = get_single_coordinate(f"南宁市{keyword}", api_key)
    if lat:
        return lat, lon


    full_address = f"你所爬取的城市{area_string.replace(' - ', '')}"
    lat, lon = get_single_coordinate(full_address, api_key)
    if not lat:
        print(f"    -> 所有策略均失败: {area_string}")

    return lat, lon

def search_poi_by_keyword(keyword, api_key):
    url = "https://restapi.amap.com/v3/place/text"
    params = {
        'key': api_key,
        'keywords': keyword,
        'city': '你所爬取的城市',
        'citylimit': True, 
        'types': '120302' 
    try:
        response = requests.get(url, params=params, timeout=10)
        data = response.json()
        if data['status'] == '1' and data['pois']:
            location = data['pois'][0]['location'] 
            lon, lat = map(float, location.split(','))
            return lat, lon
    except Exception:
        pass
    return None, None

def get_single_coordinate(address, api_key):
    geocode_url = "https://restapi.amap.com/v3/geocode/geo"
    params = {'key': api_key, 'address': address, 'city': '你所爬取的城市'}
    try:
        response = requests.get(geocode_url, params=params, timeout=10)
        data = response.json()
        if data['status'] == '1' and data['geocodes']:
            location = data['geocodes'][0]['location']
            lon, lat = map(float, location.split(','))
            return lat, lon
    except Exception:
        pass
    return None, None

def main():
    if GAODE_API_KEY == "在这里粘贴你从高德开放平台获取的Key":
        print("错误：请先在代码中配置你的高德API Key！")
        return

    df = pd.read_csv(INPUT_CSV)
    print("开始数据清洗和特征工程...")
    df['价格(元/月)'] = pd.to_numeric(df['价格(元/月)'], errors='coerce')
    df['面积(㎡)'] = pd.to_numeric(df['面积(㎡)'], errors='coerce')
    df['通勤时间(分钟)'] = pd.to_numeric(df['公交时间'].str.extract(r'([\d\.]+)')[0], errors='coerce')
    df.dropna(subset=['价格(元/月)', '面积(㎡)', '通勤时间(分钟)'], inplace=True)
    print(f"清洗后剩余 {len(df)} 条有效房源。")
    print("开始数据标准化...")
    df['价格得分'] = (df['价格(元/月)'].max() - df['价格(元/月)']) / (df['价格(元/月)'].max() - df['价格(元/月)'].min())
    df['通勤得分'] = (df['通勤时间(分钟)'].max() - df['通勤时间(分钟)']) / (df['通勤时间(分钟)'].max() - df['通勤时间(分钟)'].min())
    df['面积得分'] = (df['面积(㎡)'] - df['面积(㎡)'].min()) / (df['面积(㎡)'].max() - df['面积(㎡)'].min())
    print("计算性价比得分...")
    df['性价比得分'] = ((df['价格得分'] * WEIGHTS['price'] + df['通勤得分'] * WEIGHTS['commute'] + df['面积得分'] * WEIGHTS['area'])* 100).round(1)

    print("正在获取房源和公司地理坐标 (已采用最终优化策略)...")
    company_lat, company_lon = get_single_coordinate(COMPANY_ADDRESS, GAODE_API_KEY)
    if not company_lat:
        print("致命错误：无法获取公司坐标，程序终止。")
        return

    coords = [get_coordinates_final(row['区域'], GAODE_API_KEY) for index, row in df.iterrows()]
    coord_df = pd.DataFrame(coords, columns=['纬度', '经度'], index=df.index)
    df = pd.concat([df, coord_df], axis=1)

    original_count = len(df)
    df.dropna(subset=['纬度', '经度'], inplace=True)
    print(f"成功获取 {len(df)} / {original_count} 条房源的地理坐标。")
    
    df.to_csv(OUTPUT_CSV_SCORED, index=False, encoding='utf-8-sig')
    print(f"\n成功！已将包含性价比得分和坐标的最终数据保存到:\n{OUTPUT_CSV_SCORED}")

    print("正在生成带“蜘蛛化”功能的交互式地图...")
    m = folium.Map(location=[company_lat, company_lon], zoom_start=12)
    folium.Marker(location=[company_lat, company_lon], popup=f"<strong>公司地址</strong>", icon=folium.Icon(color='red', icon='building', prefix='fa')).add_to(m)


    marker_cluster = MarkerCluster(name='房源聚合').add_to(m)
    

    spider_group = FeatureGroupSubGroup(marker_cluster, '所有房源')
    m.add_child(spider_group)

    for idx, row in df.iterrows():
        score = row['性价比得分']
        if score > 80: color = 'green'
        elif score > 60: color = 'blue'
        elif score > 40: color = 'orange'
        else: color = 'gray'

        popup_html = f"<b>{row['标题']}</b><hr style='margin: 5px 0;'>性价比得分: <b><font color='{color}'>{score}</font></b><br>价格: {row['价格(元/月)']} 元/月<br>..."
        

        folium.CircleMarker(
            location=[row['纬度'], row['经度']],
            radius=5, color=color, fill=True, fill_color=color,
            popup=folium.Popup(popup_html, max_width=300)
        ).add_to(spider_group)
        
    m.save(OUTPUT_MAP_HTML)
    print(f"成功！已将交互式地图保存到:\n{OUTPUT_MAP_HTML}")


if __name__ == '__main__':
    main()

开始数据清洗和特征工程...
清洗后剩余 413 条有效房源。
开始数据标准化...
计算性价比得分...
正在获取房源和公司地理坐标 (已采用最终优化策略)...
成功获取 413 / 413 条房源的地理坐标。

成功！已将包含性价比得分和坐标的最终数据保存到:
E:\xiangmu\zufang\for_tableau.csv
正在生成带“蜘蛛化”功能的交互式地图...
成功！已将交互式地图保存到:
E:\xiangmu\zufang\rent_map.html
